In [ ]:
!git clone https://github.com/Hzzone/pytorch-openpose
%cd pytorch-openpose

In [ ]:
import cv2
import matplotlib.pyplot as plt
import copy
import numpy as np

from src import model
from src import util
from src.body import Body
from src.hand import Hand

In [ ]:
from glob import glob
import os

In [ ]:
os.makedirs("/kaggle/working/OpenPoseDataet/test/test")
os.makedirs("/kaggle/working/OpenPoseDataet/train/train")

In [ ]:
## set Pretrained Models

In [ ]:
!cp /kaggle/input/openpose-pretrained-models/body_pose_model.pth model
!cp /kaggle/input/openpose-pretrained-models/hand_pose_model.pth model

In [ ]:
body_estimation = Body('model/body_pose_model.pth')
hand_estimation = Hand('model/hand_pose_model.pth')

In [ ]:
from tqdm import tqdm

In [ ]:
image = plt.imread('/kaggle/input/woa7015dataset/WOA7015Dataset/Valid/Hand/12_Row42Slice0/11.png')
plt.imshow(image)

In [ ]:
frames_list = glob('/kaggle/input/7015-dataset/*/*/*')
pics_list = glob('/kaggle/input/7015-dataset/*/*/*')
pics_list = set(pics_list)
for framesPath in frames_list:
    saveFramesPath = framesPath.replace("/kaggle/input/7015-dataset","/kaggle/working/OpenPoseDataet")
    picsList = set(glob(framesPath + "/*"))
    if not os.path.exists(saveFramesPath):
        os.makedirs(saveFramesPath)
    for picPath in tqdm(picsList):
        savePicturePath = picPath.replace("/kaggle/input/7015-dataset","/kaggle/working/OpenPoseDataet")

        image = cv2.imread(picPath)
        zero_image = np.zeros(image.shape, dtype=np.uint8)
        candidate, subset = body_estimation(image)
        zero_image = util.draw_bodypose(zero_image, candidate, subset)

        hands_list = util.handDetect(candidate, subset, image)
        all_hand_peaks = []
        for x, y, w, is_left in hands_list:
            peaks = hand_estimation(image[y:y+w, x:x+w, :])
            peaks[:, 0] = np.where(peaks[:, 0]==0, peaks[:, 0], peaks[:, 0]+x)
            peaks[:, 1] = np.where(peaks[:, 1]==0, peaks[:, 1], peaks[:, 1]+y)
            all_hand_peaks.append(peaks)
        zero_image = util.draw_handpose(zero_image, all_hand_peaks)
        plt.imsave(savePicturePath, zero_image)